# Docker

## Contenedores

### Hello world

Ejecutar el primer contenedor tipo Hello world con el comando `docker run hello-world`

In [ ]:
!docker run hello-world

Como no tenemos el contenedor guardado en local, docker lo descarga de docker hub. Si ahora volvemos a ejecutar el contenedor, ya no aparecerá el primer mensaje, en el que indica que se está descargando

In [ ]:
!docker run hello-world

Para ver los contenedores que están corriendo ejecutar `docker ps`

In [ ]:
!docker ps

Comom vemos no hay ningún contenedor abierto. Pero sin embargo, si ejecutamos `docker ps -a` (`all`) vemos que si aparecen

In [ ]:
!docker ps -a

Vemos que aparecen dos contenedores llamados `hello-world` que son los dos que hemos ejecutado antes. Por tanto cada vez que ejecutamos el comando `run`, docker crea un nuevo contenedor, no ejecuta uno que ya exista

Si queremos tener más información de uno de los dos contenedores podemos ejecutar `docker inspect <id>`, donde `<id>` corresponde a la ID del docker que se ha mostrado en la lista anterior

In [ ]:
!docker inspect ID

Como acordarnos de IDs es complicado para nosotros, docker asigna nombres a los contenedores para facilitarnos la vida. Así en la lista anterior, la última columna corresponde al nombre que ha asignado docker a cada contenedor, de modo que si ahora ejecutamos `docker inspect <name>` obtendremos la misma información que con la ID

Vuelvo a ejecutar `docker ps -a` para volver a ver la lista

In [ ]:
!docker ps -a

Y ahora ejecuto `docker inspect <name>` para ver la información del contenedor

In [ ]:
!docker inspect NAME

Pero por qué con `docker ps` no vemos ningún contenedor y con `docker ps -a` sí. Esto es porque `docker ps` solo muestra los contenedores que están corriendo, mientras que `docker ps -a` muestra todos los contenedores, los que están corriendo y los que están apagados

Podemos crear un contenedor asignándole un nombre nosotros mediante el comando `docker run --name <name> hello-world`

In [ ]:
!docker run --name hello_world hello-world

Esto será más cómodo para nosotros, ya que podremos controlar nosotros los nombres de los contenedores

Si ahora queremos crear otro contenedor con el mismo nombre no podremos, porque docker no permite que se dupliquen los nombres de los contenedores. De modo que si queremos renombrar el contenedor podemos usar el comando `docker rename <old name> <new name>`

In [ ]:
!docker rename hello_world hello_world2

Tenemos ahora un montón de contenedores iguales, pero con el mismo nombre. Así que si queremos borrar alguno tenemos que usar el comando `docker rm <id>` ó `docker rm <name>`

In [ ]:
!docker rm hello_world2

Si volvemos a ver la lista de contenedores, el contenedor `hello_world2` ya no estará

In [ ]:
!docker ps -a

Si queremos borrar todos los contenedores, podemos hacerlo uno a uno, pero como es muy pesado, podemos borrar todos mediante el comando `docker container prune`. Este comando elimina solo los contenedores que estén parados

In [ ]:
!docker container prune

Docker pregunta si estás seguro, y si le dices que sí, borra todos. Si ahora listo todos los contenedores no aparece ninguno

In [ ]:
!docker ps -a

### El modo iteractivo

Vamos a ejecutar un ubuntu mediante el comando `docker run ubuntu`

In [ ]:
!docker run ubuntu

Como vemos ahora ha tardado más en descargar. Si listamos los contenedores mediante el comando `docker ps` vemos que no aparece el contenedor que acabamos de crear, es decir, no está corriendo

In [ ]:
!docker ps

Listamos ahora todos los contenedores

In [ ]:
!docker ps -a

Vemos que el estado del contenedor es `Exited (0)`

Si nos fijamos en comando del contenedor aparece `bash` y junto al estado `Exited (0)` nos indica que ha arrancado Ubuntu, ha ejecutado su *bash*, ha terminado la ejecución y ha devuelto un 0. Esto pasa porque al bash de Ubuntu no se le ha dicho nada que hacer. Para solucionar esto, ahora vamos a ejecutar el contenedor mediante el comando `docker run -it ubuntu`, con `it` lo que le estamos indicando es que lo queremos ejecutar en modo iterativo

In [ ]:
!docker run -it ubuntu

Ahora vemos que estamos dentro del bash de ubuntu. Si ejecutamos el comando `cat /etc/lsb-release` podemos ver la distribución de Ubuntu

In [ ]:
!cat /etc/lsb-release

Si abrimos otra terminal y vemos la lista de contenedores, ahora si aparecerá el contenedor corriendo Ubuntu

In [ ]:
!docker ps

Vemos el contenedor con Ubuntu y en su estado podemos ver `UP`

Si vemos ahora la lista de todos los contenedores, veremos que aparecen los dos contenedores con Ubuntu, el primero apagado y el segundo el que está corriendo

In [ ]:
!docker ps -a

Si volvemos a la terminal donde teníamos Ubuntu corriendo dentro de un docker, si escribimos ``exit`` saldremos de Ubuntu.

In [ ]:
!exit

Si ejecutamos ``docker ps`` el contenedor ya no aparece

In [ ]:
!docker ps

Pero si ejecuto ``docker ps -a`` sí que aparece. Esto quiere decir que el contenedor se apagó

In [ ]:
!docker ps -a

Esto ocurre porque al escribir ``exit``, en realidad lo estamos escribiendo en la consola del bash de Ubuntu, lo que signifique que estamos terminando el proceso bash de Ubuntu.

### Ciclo de vida de un contenedor

En docker, cuando el proceso principal de un contenedor se termina, se apaga el contenedor. Dentro de un contenedor pueden ejecutarse varios procesos, pero solo cuando se termina el proceso principal se apaga el contenedor

Por tanto, si queremos corres un contenedor que no se apague cuando finalice un proceso, debemos hacer que su proceso principal no se termine. En este caso, que no finalice bash

Si queremos ejecutar un contenedor con ubuntu, pero que no finalice cuando termine el proceso de bash lo podemos hacer de la siguiente manera

In [ ]:
!docker run --name alwaysup -d ubuntu tail -f /dev/null

Lo que hacemos es primero darle el nombre ``alwaysup``, en segundo lugar pasarle la opción ``-d`` (``detach``) para que el contenedor se ejecute en segundo plano y por último le decimos el proceso principal que queremos que se ejecute en el contenedor, que en este caso es ``tail -f /dev/null`` que equivale a un comando ``nop``

Esto nos devolverá la ID del contenedor, pero no estaremos dentro de ubuntu como pasaba antes

Si ahora vemos la lista de contenedores que se están ejecutando aparece el contenedor que acabamos de crear

In [ ]:
!docker ps

Como ya tenemos un contenedor corriendo siempre, podemos conectarnos al el mediante el comando ``exec``. Le decimos el nombre o la ID del contenedor y le pasamos el proceso que queremos que se ejecuta. Además pasamos la opción ``-it`` para decirle que sea iteractivo

In [ ]:
!docker exec -it alwaysup bash

Ahora volvemos a estar dentro de ubuntu. Si ejecutamos el commando ``ps -aux`` podemos ver una lista de los procesos que se están ejecutando dentro de ubuntu. Vemos solo tres procesos, el ``ps -aux``, el ``bash`` y el ``tail -f /dev/null``

Este contenedor va a estar siempre encendido mientras el proceso ``tail -f /dev/null`` siga corriendo

Si salimos del contenedor con el comando ``exit`` y ejecutamos el comando ``docker ps`` vemos que el contenedor sigue encendido

In [ ]:
!exit

In [ ]:
!docker ps

Para poder finalizar el proceso y poder apagar el contenedor debemos saber la id del proceso, para ello ejecutamos el comando ``docker inspect <name>``

In [ ]:
!docker inspect alwaysup

Pero esto nos da un json enorme, por lo que filtramos de la siguiente manera

In [ ]:
!docker inspect --format '{{.State.Pid}}' alwaysup

Esto nos devuelve la id del prceso principal del contenedor, por lo que mediante el comando ``kill <pid>`` podemos finalizar el proceso, por lo que apagará el contenedor

In [ ]:
!kill <pid>

Si ahora volvemos a listar los contenedores encendidos ya no aparece el contenedor con Ubuntu

In [ ]:
!docker ps

Y si listamos todos los contenedores, aparece el contenedor con Ubuntu, y su estado ``Exited``

Vamos a volver a lanzar el contenedor con Ubuntu para ver otra manera de parar un contenedor, para ello ejecutamos el comando ``docker start <name>``

In [ ]:
!docker start alwaysup

Si volvemos a listar los contenedores que están corriendo volveremos a ver el contenedor con ubuntu. Ahora, para volver a pararlo podemos usar el comando ``docker stop <name>``

In [ ]:
!docker stop alwaysup

Si volvemos a listar los contenedores, ya no aparecerá el contenedor con ubuntu corriendo

### Exponer contenedores al mundo exterior

Vamos a crear un nuevo contenedor con un servidor

In [ ]:
!docker run -d --name proxy nginx

Esto crea un servidor, vamos a volver a listar los contenedores que están corriendo

In [ ]:
!docker ps

Ahora aparece una nueva columna con el puerto, y nos dice que el servidor que acabamos de crear está en el puerto ``80`` bajo el protocolo ``tcp``. Si abrimos un navegador e intentamos conectarnos al servidor mediante ``http://localhost:80`` no conseguimos conectar. Esto es porque cada contenedor tiene su propia interfaz de red. Es decir, el servidor está escuchando en el puerto ``80`` del contenedor, pero nosotros estamos intentando conectar al puerto ``80`` del host

Paramos el contenedor para relanzarlo de otra forma

In [ ]:
!docker stop proxy

Si listamos los contenedores no aparece corriendo

In [ ]:
!docker ps

Lo borramos para volver a crearlo

In [ ]:
!docker rm proxy

Si listamos todos los contenedores ya no está

In [ ]:
!docker ps -a

Para volver a crear el contenedor con el servidor y poderlo ver desde el host, tenemos que usar la opción ``-p`` (``publish``), indicando en primer lugar el puerto en el que queremos verlo en el host y a continuación el puerto del contenedor, es decir, ``-p <ip host>:<ip conteiner>``

In [ ]:
!docker run -d --name proxy -p 8080:80 nginx

Si ahora vamos a un navegador e introducimos ``https://localhost:8080`` podremos acceder al servidor del contenedor

Si ademas listamos los contenedores, en la columna ``PORTS`` indica ``0.0.0.0:8080->80/tcp``, lo que nos ayuda a ver la relación de puertos

Para ver los logs del contenedor, mediante el comando ``docker logs <name>`` puedo ver los logs del contenedor

In [ ]:
!docker logs proxy

Ahora puedo ver todas las peticiones que se le han hecho al servidor. Pero si quiero ver los logs en tiempo real, mediante ``docker logs -f <name>`` lo puedo hacer

In [ ]:
!docker logs -f proxy

Ahora puedo ver los logs en tiempo real. Para salir introducir ``CTRL+C``

Como puede llegar un momento en el que haya muchos logs, si solo quieres los últimos logs, mediante la opción ``--tail <num>`` puedo ver los últimos ``<num>`` logs. Si añado la opción ``-f`` estaremos viendo siempre los últimos ``<num>`` logs

In [ ]:
!docker logs --tail 10 -f proxy

## Datos en Docker

### Bind mounts

Vamos a crear una carpeta para trabajar en esta parte

In [ ]:
!mkdir dockerdata

Ahora vamos a crear una base de datos de mongodb

In [ ]:
!docker run -d --name db mongo

Si listamos los contenedores aparece este que acabamos de crear

In [ ]:
!docker ps

Ahora ejecutamos un bash en dicho contenedor

!docker exec -it db bash

En el contenedor, puedo crear una nueva carpeta que se llame ``dockerfolder``

In [ ]:
!mkdir dockerfolder

Si listamos los archivos aparecerá la nueva carpeta

In [ ]:
!ls

Si salimos del contenedor y lo borramos 

In [ ]:
!exit

!docker rm -f db

Si listamos todos los contenedores ya no aparece el último que hemos creado

In [ ]:
!docker ps -a

Vamos a volver a hacer todo, pero primero vamos a crear una carpeta en el host en la que compartiremos los datos con el contenedor

In [ ]:
!mkdir mongodata

Vemos qué hay dentro de mongodata

In [ ]:
!ls mongodata/

Vemos que no hay nada. Ahora obtenemos nuestra ruta absoluta

In [ ]:
!pwd

Volvemos a crear el contenedor pero añadiendo la opción ``-v`` (``bind mount``). A continuación se añade la ruta absoluta de la carpeta del host y la ruta absoluta de la carpeta en el contenedor, ``-v <host path>:<container path>``

In [ ]:
!docker run -d --name db -v <host path>:/data/db mongo

Entramos al contenedor, listamos los archivos y ya no aparece la carpeta que habíamos creado

In [ ]:
!docker exec -it db bash

Vamos al directorio del contenedor que hemos compartido

In [ ]:
!cd /data/db

Creamos un nuevo archivo

In [ ]:
!touch bindfile.txt

Salimos del contenedor

In [ ]:
!exit

Vamos a la carpeta compartida del host

In [ ]:
!cd <host folder>

Listamos los archivos y vemos que está el archivo que hemos creado antes

In [ ]:
!ls <host folder>/

Pero es más, si borramos el contenedor, el archivo sigue ahí

In [ ]:
!docker rm -f db

In [ ]:
!ls <host folder>/

Si vuelvo a crear un contenedor con mongo, compartiendo las carpetas, todos los archivos estarán en el contenedor

In [ ]:
!docker run -d --name db -v <host folder>:/data/db mongo

In [ ]:
!docker exec -it db bash

In [ ]:
!ls /data/db

### Volúmenes

Los volúmenes se crearon como una evolución de los ``bind mounts`` para dar más seguridad. Podemos listar todos los volúmenes de docker mediante ``docker volume ls``

In [ ]:
!docker volume ls

Vamos a crear un nuevo volumen para el contenedor de mongodb, para ello usamos el comando ``docker volume create <volume name>``

In [ ]:
!docker volume create datadb

Si volvemos a listar los volúmenes aparecerá el que acabamos de crear

In [ ]:
!docker volume ls

Sin embargo no aparece como una carpeta en el sistema de archivos del host

In [ ]:
!ls

Vamos a volver a crear un contenedor usando el volumen que acabamos de crear, para ello, primero borramos el contenedor de mongodb que habíamos creado

In [ ]:
!docker rm -f db

Ahora creamos el contenedor con el volumen que acabamos de crear con la opción ``--mount``, indicando el volumen fuente mediante ``src=<volume name>`` (si el volumen no existiese, docker lo crearía), a continuación el destino separado por una ``,``, ``dst=<container path>``, es decir ``--mount src=<volume name>,dst=<container path>``

In [ ]:
!docker run -d --name db --mount src=datadb,dst=/data/db mongo

Una vez creado podemos ver los volúmenes del contenedor mediante el comando ``inspect`` y filtrando por 

In [ ]:
!docker inspect --format '{{.Mounts}}' db

Vemos que el primer volumen se llama ``<datadb>`` y ademas podemos ver la ruta dónde está guardado, en este caso en ``<volume path>``. Hacemos lo mismo que antes, nos metemos en el contenedor, creamos un archivo en la ruta del volumen, salimos y vemos en el host si se ha creado

In [ ]:
!docker exec -it db bash

In [ ]:
!cd /data/db

In [ ]:
!touch /data/db/volumefile.txt

In [ ]:
!exit

In [ ]:
!ls <volume path>

Está el archivo creado

### Insertar y extraer archivos de un contenedor

Primero vamos a crear un archivo que queremos copiar dentro de un contenedor

In [ ]:
!touch text.txt

Ahora vamos a crear un nuevo contenedor en el que queremos copiar un archivo

In [ ]:
!docker run -d --name copyfile ubuntu tail -f /dev/null

Entramos en el contenedor

In [ ]:
!docker exec -it copyfile bash

Creamos una nueva carpeta donde vamos a copiar el archivo

In [ ]:
!mkdir foldercopy

Salimos del contenedor y copiamos el archivo mediante el comando ``cp``, indicando el archivo que quiero copiar, el contenedor donde lo queremos copiar y la ruta dentro del contenedor, ``docker cp <file> <container>:<container path>``

In [ ]:
!exit

In [ ]:
!docker cp text.txt copyfile:/foldercopy

Volvemos a entrar al contenedor y comprobamos que está el archivo

In [ ]:
!docker exec -it copyfile bash

In [ ]:
!ls foldercopy

Salimos del contenedor

In [ ]:
!exit

Ahora vamos a extraer el archivo del contenedor y lo vamos a guardar en el host con otro nombre, para ello usamos el comando otra vez el comando ``cp``, pero indicando ahora el contenedor, la ruta del archivo en el contenedor y la ruta y nombre del que queremos que tenga el archivo en el host, ``docker cp <container>:<docker file path> <host file path>``

In [ ]:
!docker cp copyfile:/foldercopy fileextract.txt

Vemos que está en el host

In [ ]:
!ls

Aunque el contenedor esté parado se pueden copiar archivos